In [8]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np 
from google import genai

In [9]:
load_dotenv()

google_key = os.getenv("GOOGLE_API_KEY")

In [16]:
location = os.getcwd() + "/data"
files = os.listdir(location)

In [19]:
df = pd.DataFrame()
for file in files:
    data = pd.read_csv("data/" + file)
    df = pd.concat([df, data], axis=0)



/var/folders/4q/5_yrsp9s6s561gbs7n4b8k_00000gn/T/ipykernel_22898/700344308.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, data], axis=0)
/var/folders/4q/5_yrsp9s6s561gbs7n4b8k_00000gn/T/ipykernel_22898/700344308.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, data], axis=0)
/var/folders/4q/5_yrsp9s6s561gbs7n4b8k_00000gn/T/ipykernel_22898/700344308.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecat

In [29]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5917 entries, 0 to 130
Data columns (total 35 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [35]:
dd_df = df[df['Text'].duplicated() == False]

In [37]:
dd_df.shape, df.shape

((5640, 35), (5917, 35))

'NYSC! Here I Come!\n.\n📸: @CelinaSesi https://t.co/ykAx34AxBg'

In [47]:
client = genai.Client(api_key=google_key)

response = client.models.generate_content(model="gemini-2.5-flash", contents=contents+df['Text'][0].values[0])

print(response.text)

Positive


In [55]:
dd_df = dd_df.reset_index(drop=True)

In [80]:
client = genai.Client(api_key=google_key)
def sentiment(text):
    contents = "I need you to help extract sentiments from this tweets. classify them into Positive, negative or neutral. Just return the sentiment as the text alone"
    response = client.models.generate_content(model="gemini-2.5-flash", contents=contents+text)
    return response.text

for i in range(80, dd_df.shape[0]):
    dd_df['Sentiment']= sentiment(dd_df['Text'][i])
    print(i)
    


80
81
82
83
84
85
86
87
88
89
90


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'}, 'quotaValue': '10'}]}, {'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '26s'}]}}

In [ ]:
import time
from google.api_core.exceptions import ClientError

def get_sentiment(text_content):
    contents_instruction = "I need you to help extract sentiments from this tweet. Classify it into Positive, negative or neutral. Just return the sentiment as the text alone."
    # Use a loop with retry logic for API calls
    max_retries = 5
    for attempt in range(max_retries):
        try:
            response = client.models.generate_content(
                model="gemini-2.5-flash",
                contents=[contents_instruction, text_content]
            )
            return response.text.strip()
        except ClientError as e:
            # Check if it's a quota error (429)
            if e.status_code == 429:
                print(f"Quota exceeded (429) for text: '{text_content}'. Retrying in {2**(attempt+1)} seconds... (Attempt {attempt + 1}/{max_retries})")
                time.sleep(2**(attempt+1)) # Exponential backoff: 2, 4, 8, 16, 32 seconds
            else:
                # Re-raise other ClientErrors immediately
                print(f"An unexpected ClientError occurred: {e}. Returning 'Error'.")
                return "Error"
        except Exception as e:
            # Catch any other unexpected errors
            print(f"An unexpected error occurred for text: '{text_content}' - {e}. Returning 'Error'.")
            return "Error"
    print(f"Failed to get sentiment after {max_retries} attempts for text: '{text_content}'.")
    return "Failed" # Return 'Failed' if all retries are exhausted


In [96]:
dd_df.loc[901, 'Sentiment']

'Positive'

In [ ]:
sentiments = []
for i in range(0, dd_df.shape[0]):
    sentiment = get_sentiment(dd_df['Text'][i])
    sentiments.append(sentiment)

    # Wait every 8 texts to avoid hitting quota
    if (i + 1) % 8 == 0:
        print(f"Processed {i + 1} texts, sleeping for 30 seconds to avoid quota issues.")
        time.sleep(30)  # Adjust sleep time based on quota limits
    if (i + 1) % 8 == 0:
        dd_df.loc[:i, 'Sentiment'] = sentiments
        dd_df.to_csv('sentiment_partial.csv', index=False)


TypeError: catching classes that do not inherit from BaseException is not allowed